<a href="https://colab.research.google.com/github/trtrtz/Big-Data-Project/blob/main/ds_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Airline Price Prediction Project

**Code Author:** Ting Zhou

**Date:** October 2025  
**Class:** Data Science

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random

csv_path = '/content/drive/MyDrive/itineraries_sample.csv'

flight_sample = pd.read_csv(csv_path)
flight_sample.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,0d654f1c4884897aff1c2ffb968e6e40,2022-10-05,2022-11-08,ORD,SFO,KAU4AKBS,PT7H57M,1,True,False,...,1667969040||1667981100,2022-11-08T20:44:00.000-08:00||2022-11-09T00:0...,LAX||SFO,ORD||LAX,United||United,UA||UA,Boeing 757-300||Airbus A320,16560||5340,1745||339,coach||coach
1,b0c2b8e7f41fd456ca95b41e00d9ce56,2022-10-05,2022-11-08,PHL,LAX,UAUTC2BC,PT9H19M,0,True,False,...,1667910900||1667936940,2022-11-08T07:35:00.000-05:00||2022-11-08T11:4...,ATL||ONT,PHL||ATL,Delta||Delta,DL||DL,Boeing 737-900||Airbus A321,7500||17100,667||1897,coach||coach
2,6b2e2224f5e8cec5f02e4a1221d405b9,2022-10-05,2022-11-11,PHL,DEN,PI7QBEL1,PT14H5M,0,False,False,...,1668185040||1668208320||1668231120,2022-11-11T11:44:00.000-05:00||2022-11-11T18:1...,BOS||JFK||DEN,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Airbus A320||Airbus A320||Airbus A320,4620||4500||16320,280||186||1632,coach||coach||coach
3,844531694368ca0ccd15d3fad4bf230d,2022-10-05,2022-11-10,DFW,SFO,TAA2JQBS,PT6H17M,0,True,False,...,1668083700||1668102120,2022-11-10T06:35:00.000-06:00||2022-11-10T09:4...,IAH||SFO,DFW||IAH,United||United,UA||UA,Boeing 737-700||Boeing 777,4200||15420,233||1641,coach||coach
4,bac03eeb6f097dd86d66d1c5438778fe,2022-10-05,2022-11-10,LAX,EWR,QAVTA0MC,PT8H39M,0,False,False,...,1668106680||1668123840,2022-11-10T13:58:00.000-05:00||2022-11-10T18:4...,ATL||EWR,ONT||ATL,Delta||Delta,DL||DL,Airbus A321||Boeing 737-800,13980||7680,1897||756,coach||coach


In [ ]:
import pandas as pd

# classifying competitions
# columns used: startingAirport, destinationAirport, segmentsAirlineCode

#flights per route per airline
route_airline = (
    flight_sample
    .groupby(['startingAirport', 'destinationAirport', 'segmentsAirlineCode'])
    .size()
    .reset_index(name='count')
)

# market share within each route
route_airline['total_route'] = (
    route_airline.groupby(['startingAirport', 'destinationAirport'])['count']
    .transform('sum')
)
route_airline['share'] = route_airline['count'] / route_airline['total_route']

# Herfindahl–Hirschman Index (HHI)
hhi = (
    route_airline
    .groupby(['startingAirport', 'destinationAirport'])['share']
    .apply(lambda s: (s**2).sum())
    .reset_index(name='HHI')
)

# classifying market type
def classify(hhi_val):
    if hhi_val > 0.25:
        return 'Monopoly'
    elif hhi_val > 0.15:
        return 'Oligopoly'
    else:
        return 'Competitive'

hhi['MarketType'] = hhi['HHI'].apply(classify)

flight_sample = flight_sample.merge(hhi, on=['startingAirport', 'destinationAirport'], how='left')

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

print(hhi.head())


  startingAirport destinationAirport       HHI MarketType
0             ATL                BOS  0.157388  Oligopoly
1             ATL                CLT  0.249657  Oligopoly
2             ATL                DEN  0.167441  Oligopoly
3             ATL                DFW  0.163737  Oligopoly
4             ATL                DTW  0.155175  Oligopoly


In [ ]:
# distribution of market types
market_counts = hhi['MarketType'].value_counts(normalize=True) * 100

print(market_counts.round(2).to_string())

MarketType
Oligopoly      67.52
Monopoly       17.09
Competitive    15.38


Preliminary data integreity check:

1.   uniqueness, logical constraint, time integrity
2.   清單項目



In [ ]:
import pandas as pd
import numpy as np

# ---------- Helpers ----------
BOOL_COLS = ['isBasicEconomy','isRefundable','isNonStop']
SEGMENT_COLS = [
    'segmentsDepartureTimeEpochSeconds','segmentsDepartureTimeRaw',
    'segmentsArrivalTimeEpochSeconds','segmentsArrivalTimeRaw',
    'segmentsArrivalAirportCode','segmentsDepartureAirportCode',
    'segmentsAirlineName','segmentsAirlineCode',
    'segmentsEquipmentDescription','segmentsDurationInSeconds',
    'segmentsDistance','segmentsCabinCode'
]

def _to_bool(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (bool, np.bool_)): return bool(x)
    if isinstance(x, (int, np.integer)) and x in (0,1): return bool(x)
    if isinstance(x, str):
        s = x.strip().lower()
        if s in ('1','true','t','yes','y'): return True
        if s in ('0','false','f','no','n'): return False
    return np.nan

def _count_legs(val):
    if pd.isna(val) or val=='':
        return 0
    return str(val).count('||') + 1

def _median(x):
    try: return np.nanmedian(x)
    except: return np.nan

# uniqueness check
def check_duplicates(df):
    total = len(df)
    uniq = df['legId'].nunique(dropna=False)
    dups = total - uniq
    print(f"[Duplicates] rows={total:,}, unique_legId={uniq:,}, duplicates={dups:,}")
    return {"rows": total, "unique_legId": uniq, "duplicates": dups}

# value check (check if entry type is valid)
def check_fares_bools_seats(df):
    base = pd.to_numeric(df['baseFare'], errors='coerce')
    total = pd.to_numeric(df['totalFare'], errors='coerce')

    v_base_pos = np.sum(~(base > 0))
    v_order    = np.sum(~(base <= total))
    print(f"[Fares] invalid base>0: {v_base_pos:,} | base<=total violated: {v_order:,}")

    out = {"fare_invalid_base>0": int(v_base_pos), "fare_order_violations": int(v_order)}

    # columns with boolean values
    bool_df = df[BOOL_COLS].applymap(_to_bool)
    invalid_bool = bool_df.isna().sum().to_dict()
    print("[Booleans] invalid counts:", invalid_bool)
    out["invalid_booleans"] = invalid_bool

    # seats(positive)
    seats = pd.to_numeric(df['seatsRemaining'], errors='coerce')
    neg = np.sum(seats < 0)
    nonint = np.sum(~seats.isna() & (seats % 1 != 0))
    desc = seats.describe()
    print(f"[Seats] negative={neg:,}, non-integer={nonint:,}")
    print(desc.to_string())
    out.update({"seats_negative": int(neg), "seats_nonint": int(nonint), "seats_desc": desc.to_dict()})
    return out

# time integrity(check for timeline violation)
def check_times(df):
    sd = pd.to_datetime(df['searchDate'], errors='coerce')
    fd = pd.to_datetime(df['flightDate'], errors='coerce')

    unparsable_sd = sd.isna().sum()
    unparsable_fd = fd.isna().sum()
    print(f"[Time] unparsable searchDate={unparsable_sd:,}, flightDate={unparsable_fd:,}")

    order_viol = np.sum(sd > fd)
    print(f"[Time] searchDate > flightDate violations: {order_viol:,}")

    book_win = (fd - sd).dt.days
    bw_desc = book_win.describe()
    neg = np.sum(book_win < 0)
    huge = np.sum(book_win > 365)
    print(f"[Booking window] negative={neg:,}, >365d={huge:,}")
    print(bw_desc.to_string())

    return {
        "unparsable_searchDate": int(unparsable_sd),
        "unparsable_flightDate": int(unparsable_fd),
        "order_violations": int(order_viol),
        "booking_window_desc": bw_desc.to_dict(),
        "booking_window_negative": int(neg),
        "booking_window_gt365": int(huge)
    }

# segment consistency (making sure they're connected)
def check_segments(df):
    leg_counts = {}
    for c in SEGMENT_COLS:
        if c in df.columns:
            leg_counts[c] = df[c].apply(_count_legs)
    legc = pd.DataFrame(leg_counts)

    # mismatch rows: max != min across segment fields (ignore missing columns)
    if legc.shape[1] == 0:
        print("[Segments] No segment columns found.")
        return {"segment_mismatch_rows": None}

    mismatch = (legc.max(axis=1) != legc.min(axis=1))
    mismatch_pct = 100 * mismatch.mean()
    print(f"[Segments] mismatched leg counts: {mismatch.sum():,} rows ({mismatch_pct:.2f}%)")

    # endpoint coherence (only if codes exist)
    endpoint_mismatch = None
    if 'segmentsDepartureAirportCode' in df.columns and 'segmentsArrivalAirportCode' in df.columns:
        dep_first = df['segmentsDepartureAirportCode'].str.split(r'\|\|').str[0]
        arr_last  = df['segmentsArrivalAirportCode'].str.split(r'\|\|').str[-1]
        m1 = (dep_first != df['startingAirport']).fillna(True)
        m2 = (arr_last  != df['destinationAirport']).fillna(True)
        endpoint_mismatch = {"start_code_mismatch": int(m1.sum()), "dest_code_mismatch": int(m2.sum())}
        print(f"[Segments] endpoint mismatches -> start:{m1.sum():,}, dest:{m2.sum():,}")

    return {
        "segment_mismatch_rows": int(mismatch.sum()),
        "endpoint_mismatch": endpoint_mismatch
    }

# make sure duration&distance align with the sum of segments
def check_duration_distance(df):
    out = {}

    # Nonstop coherence
    legs = df.get('segmentsDepartureTimeRaw', pd.Series(index=df.index)).apply(_count_legs)
    nonstop = df['isNonStop'].apply(_to_bool)
    contradiction = np.sum((nonstop == True) & (legs > 1))
    print(f"[Nonstop] contradictions (nonstop but legs>1): {contradiction:,}")
    out["nonstop_contradictions"] = int(contradiction)

    # Totals vs segment sums (if available)
    if 'segmentsDurationInSeconds' in df.columns and 'travelDuration' in df.columns:
        # normalize travelDuration to seconds if it's like "hh:mm"
        td = df['travelDuration'].astype(str)
        td_sec = pd.to_timedelta(td.where(td.str.contains(':'), np.nan), errors='coerce').dt.total_seconds()

        seg_sum = df['segmentsDurationInSeconds'].apply(
            lambda s: np.nansum(pd.to_numeric(str(s).split('||'), errors='coerce')) if pd.notna(s) else np.nan
        )
        delta = (td_sec - seg_sum)
        tol = 0.1 * seg_sum  # 10% tolerance
        bad = np.sum(~delta.isna() & (delta.abs() > tol.fillna(np.inf)))
        print(f"[Duration] total vs segments diff >10%: {bad:,}")
        out["duration_diff_gt10pct"] = int(bad)

    if 'segmentsDistance' in df.columns and 'totalTravelDistance' in df.columns:
        ttd = pd.to_numeric(df['totalTravelDistance'], errors='coerce')
        segd = df['segmentsDistance'].apply(
            lambda s: np.nansum(pd.to_numeric(str(s).split('||'), errors='coerce')) if pd.notna(s) else np.nan
        )
        dlt = (ttd - segd)
        tol = 0.1 * segd
        bad = np.sum(~dlt.isna() & (dlt.abs() > tol.fillna(np.inf)))
        print(f"[Distance] total vs segments diff >10%: {bad:,}")
        out["distance_diff_gt10pct"] = int(bad)

    return out

# check if there's abnormal pricing based on featuers
def check_price_coherence(df):
    out = {}
    total = pd.to_numeric(df['totalFare'], errors='coerce')

    # Refundable uplift
    ref = df['isRefundable'].apply(_to_bool)
    med_ref = _median(total[ref==True])
    med_non = _median(total[ref==False])
    uplift = med_ref - med_non
    print(f"[Refundable] median(ref)={med_ref:.2f}, median(non)={med_non:.2f}, uplift={uplift:.2f}")
    out["refundable_median_uplift"] = float(uplift)

    # Nonstop premium
    ns = df['isNonStop'].apply(_to_bool)
    med_ns  = _median(total[ns==True])
    med_con = _median(total[ns==False])
    prem = med_ns - med_con
    print(f"[Nonstop] median(nonstop)={med_ns:.2f}, median(connecting)={med_con:.2f}, premium={prem:.2f}")
    out["nonstop_median_premium"] = float(prem)

    # Cabin ladder (if any)
    if 'segmentsCabinCode' in df.columns:
        ladder = total.groupby(df['segmentsCabinCode']).median().sort_values()
        print("[Cabin] median ladder:\n", ladder.to_string())
        out["cabin_median_ladder"] = ladder.to_dict()

    return out

# missing data
def check_missingness(df):
    miss = df.isna().mean().sort_values(ascending=False) * 100
    print("[Missingness %] top columns:")
    print(miss.head(15).round(2).to_string())
    return {"missing_pct": miss.to_dict()}

# market type distribution
def check_hhi(df):
    # airline competition at route level
    grp = (df.groupby(['startingAirport','destinationAirport','segmentsAirlineCode'])
             .size().reset_index(name='count'))
    grp['route_total'] = grp.groupby(['startingAirport','destinationAirport'])['count'].transform('sum')
    grp['share'] = grp['count'] / grp['route_total']
    hhi = grp.groupby(['startingAirport','destinationAirport'])['share'].apply(lambda s: (s**2).sum()).reset_index(name='HHI')

    def classify(x):
        if x > 0.25: return 'Monopoly'
        elif x > 0.15: return 'Oligopoly'
        else: return 'Competitive'
    hhi['MarketType'] = hhi['HHI'].apply(classify)

    pct = (hhi['MarketType'].value_counts(normalize=True) * 100).round(2)
    print("[HHI] % by MarketType:\n", pct.to_string())
    return {"hhi": hhi, "markettype_pct": pct.to_dict()}

# pricing outliers
def check_outliers(df):
    total = pd.to_numeric(df['totalFare'], errors='coerce')
    q1, q3 = np.nanpercentile(total, [25, 75])
    iqr = q3 - q1
    lo, hi = q1 - 3*iqr, q3 + 3*iqr # flagging price outside this range
    outliers = np.sum((total < lo) | (total > hi))
    print(f"[Outliers] totalFare outside [Q1-3*IQR, Q3+3*IQR]: {outliers:,}")
    return {"price_outliers": int(outliers), "bounds": {"low": float(lo), "high": float(hi)}}

#
def pre_checks(flight_sample: pd.DataFrame):
    results = {}
    print("\n1) Duplicates")
    results["duplicates"] = check_duplicates(flight_sample)

    print("\n2) Fares, Booleans, Seats Validity")
    results["fares_bools_seats"] = check_fares_bools_seats(flight_sample)

    print("\n3) Time Integrity")
    results["time"] = check_times(flight_sample)

    print("\n4) Segment Consistency")
    results["segments"] = check_segments(flight_sample)

    print("\n5) Duration & Distance Alignment")
    results["dur_dist"] = check_duration_distance(flight_sample)

    print("\n6) Price–Feature Coherence ")
    results["price_coherence"] = check_price_coherence(flight_sample)

    print("\n7) Missingness of entries")
    results["missingness"] = check_missingness(flight_sample)

    print("\n8) Coverage & HHI ")
    results["hhi"] = check_hhi(flight_sample)

    print("\n9) Outliers")
    results["outliers"] = check_outliers(flight_sample)

    return results

results = pre_checks(flight_sample)



1) Duplicates
[Duplicates] rows=20,000, unique_legId=20,000, duplicates=0

2) Fares, Booleans, Seats Validity
[Fares] invalid base>0: 0 | base<=total violated: 0
[Booleans] invalid counts: {'isBasicEconomy': 0, 'isRefundable': 0, 'isNonStop': 0}
[Seats] negative=0, non-integer=0
count    20000.000000
mean         6.742750
std          2.588495
min          0.000000
25%          7.000000
50%          7.000000
75%          9.000000
max         10.000000

3) Time Integrity
[Time] unparsable searchDate=0, flightDate=0
[Time] searchDate > flightDate violations: 0
[Booking window] negative=0, >365d=0
count    20000.000000
mean        33.194850
std          4.860387
min          1.000000
25%         31.000000
50%         35.000000
75%         36.000000
max         44.000000

4) Segment Consistency


/tmp/ipython-input-2919451071.py:54: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bool_df = df[BOOL_COLS].applymap(_to_bool)


[Segments] mismatched leg counts: 564 rows (2.82%)
[Segments] endpoint mismatches -> start:424, dest:491

5) Duration & Distance Alignment
[Nonstop] contradictions (nonstop but legs>1): 0
[Duration] total vs segments diff >10%: 0
[Distance] total vs segments diff >10%: 57

6) Price–Feature Coherence 
[Refundable] median(ref)=nan, median(non)=218.60, uplift=nan
[Nonstop] median(nonstop)=158.60, median(connecting)=245.60, premium=-87.00
[Cabin] median ladder:
 segmentsCabinCode
coach                           158.600
coach||coach                    221.600
coach||first                    278.100
coach||premium coach            451.600
coach||coach||coach||coach      519.290
coach||coach||coach             521.400
premium coach||coach            672.095
coach||coach||premium coach     703.090
premium coach                   998.600
first||first                   1609.600
first                          2057.610
first||coach                   2157.600
business||business             2697.200

/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Remark: The dataset is overall clean and reliable. It includes 20,000 unique flight records with no duplicates and valid entries across fares, booleans, and seat counts. Seat availability averages around seven per flight, suggesting data were collected at reasonable booking stages.

Time information is fully consistent: all dates are valid, and every search precedes its flight. The booking window averages about 33 days, reflecting realistic traveler behavior.

Segment data are largely coherent, with only about 3% showing mismatched leg counts or minor endpoint errors—small enough not to affect overall trends. Duration and distance fields also align well: total and segment-level sums agree almost perfectly, and non-stop indicators match the number of legs.

In pricing patterns, refundable fares are mostly missing, while non-stop flights are unexpectedly cheaper than connecting ones—likely due to strong competition on direct, high-demand routes. Cabin-level pricing follows expected order, with prices rising from coach to business class.

Missingness is minimal aside from totalTravelDistance, which is absent in roughly 11% of rows. Outliers are rare (about 1.2% of fares), suggesting stable and realistic price ranges.

Market structure analysis shows that most routes are oligopolistic (about two-thirds), with fewer monopoly and competitive cases. This confirms the dataset’s bias toward major, high-traffic routes, limiting representativeness for smaller markets.

Overall, the data are clean and well-behaved, suitable for modeling price behavior. Remaining issues—minor segment mismatches, limited refundable data, and route concentration—are modest and can be handled easily in preprocessing.